In [48]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC

### Importing the data

In [139]:
data = pd.read_excel("./Data/Events Data.xlsx")
teams = pd.read_excel("./Data/Game Metadata.xlsx",sheet_name=0)
managers = pd.read_excel("./Data/Game Metadata.xlsx",sheet_name=1)
manager_history = pd.read_excel("./Data/Game Metadata.xlsx",sheet_name=2)
referees = pd.read_excel("./Data/Game Metadata.xlsx",sheet_name=3)

test = pd.read_excel("./Data/Test Data.xlsx")
data.head(10)

,season,game_week,game_id,game_date,home_team_id,away_team_id,home_goals_scored,away_goals_scored,home_formation,away_formation,home_num_injuries,away_num_injuries,referee_id
0,2012,1,efl_00001,2012/02/08,xmq321,1xyza3,1,0,four three three,four two four,0,2,brit12
1,2012,1,efl_00002,2012/02/08,ytug99,jzxz42,2,3,four five one,four three three,0,0,kope3
2,2012,1,efl_00003,2012/02/08,9hgy&,uythe,2,3,four five one,four five one,1,4,xyyu5
3,2012,1,efl_00004,2012/02/08,6re#$3,bl4a#,2,0,four five one,three four three,0,3,manc77
4,2012,1,efl_00005,2012/02/08,gyu43a,krkbr33,2,3,four four two,four five one,0,0,xhh46
5,2012,1,efl_00006,2012/02/08,l0p4ra,ju$xzs,2,3,five three two,five three two,1,0,kloh78
6,2012,1,efl_00007,2012/02/08,tq8mo,xuxa#,0,1,three four three,three four three,3,2,defrh56
7,2012,1,efl_00008,2012/02/08,p235rr,rew#ff,0,1,four two four,five three two,1,2,mjam43
8,2012,1,efl_00009,2012/02/08,xyx311,fdq34,0,1,five four one,four five one,0,0,jurg43
9,2012,1,efl_00010,2012/02/08,zd78$,gyt676,0,2,four three three,four five one,0,2,xlo9


In [3]:
teams.head(10)

,team_id,team_name,team_city,team_location,team_country,team_stadium,stadium_capacity
0,xyx311,Lloyd FC,Wakefield,"53.6833 N, 1.5059 W",England,Marvel Ground,38439
1,p235rr,Rekordmeister,Berlin,"52.5200 N, 13.4050 E",Germany,Hoofdstad Stadion,63753
2,gyt676,Lombardia,Verona,"45.4384 N, 10.9916 E",Italy,Stadio Nazionale,51268
3,krkbr33,Manzanares,Malaga,"36.7213 N, 4.4213 W",Spain,Cornella,32037
4,bl4a#,Byzantinobul,Istanbul,"41.0082 N, 28.9784 E",Turkey,Ataturk Olympic,35008
5,uythe,Hammarby,Stockholm,"59.3293 N, 18.0686 E",Sweden,Viterbi Creek,48040
6,tq8mo,Piemonte,Ravenna,"44.4184 N, 12.2035 E",Italy,Estadio Massimino,52994
7,1xyza3,Old Firm,Glasgow,"55.8642 N, 4.2518 W",Scotland,Ellis Park,39152
8,l0p4ra,Valdai,Moscow,"55.7558 N, 37.6173 E",Russia,Khabibov Stadium,39603
9,gyu43a,Europort,Oxford,"51.7520 N, 1.2577 W",England,Rose Park,39249


In [4]:
managers.head()

,manager_id,manager_name,manager_country,manager_badge,manager_age
0,feti64,Federico Ancelotti,Italy,ITA,51
1,jaon68,Janne Clarkson,Sweden,SWE,60
2,luba89,Lucas Saliba,France,FRA,50
3,joay35,John Mctominay,Scotland,SCO,44
4,geov76,Georgi Zhirkov,Russia,RUS,65


In [5]:
manager_history.head()

,season,team_id,manager_id
0,2012,xyx311,cane7
1,2013,xyx311,luba89
2,2014,xyx311,erin77
3,2015,xyx311,jaon68
4,2016,xyx311,dapp36


In [6]:
referees.head()

,referee_id,referee_name,referee_badge,referee_height,referee_country,referee_experience,referee_marital_status
0,mjam43,Michael James,EFL,165,England,20,Y
1,xyhh3,Emre Neuer,EFL,167,Germany,17,Y
2,xysd4,Giovanni Conte,EFL,162,Italy,18,Y
3,xyyu5,Cihat Eren,EFL,164,Turkey,16,N
4,xlo9,Seref Cakir,EFL,165,Turkey,23,N


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7600 entries, 0 to 7599
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   season             7600 non-null   int64 
 1   game_week          7600 non-null   int64 
 2   game_id            7600 non-null   object
 3   game_date          7600 non-null   object
 4   home_team_id       7600 non-null   object
 5   away_team_id       7600 non-null   object
 6   home_goals_scored  7600 non-null   int64 
 7   away_goals_scored  7600 non-null   int64 
 8   home_formation     7600 non-null   object
 9   away_formation     7600 non-null   object
 10  home_num_injuries  7600 non-null   int64 
 11  away_num_injuries  7600 non-null   int64 
 12  referee_id         7600 non-null   object
dtypes: int64(6), object(7)
memory usage: 772.0+ KB


In [141]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   season             760 non-null    int64 
 1   game_week          760 non-null    int64 
 2   game_id            760 non-null    object
 3   game_date          760 non-null    object
 4   home_team_id       760 non-null    object
 5   away_team_id       760 non-null    object
 6   home_formation     760 non-null    object
 7   away_formation     760 non-null    object
 8   home_num_injuries  760 non-null    int64 
 9   away_num_injuries  760 non-null    int64 
 10  referee_id         760 non-null    object
dtypes: int64(4), object(7)
memory usage: 65.4+ KB


### Featur Engineering

In [145]:
train_df = data.copy()
test_df = test.copy()
df = pd.concat([train_df,test_df])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8360 entries, 0 to 759
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season             8360 non-null   int64  
 1   game_week          8360 non-null   int64  
 2   game_id            8360 non-null   object 
 3   game_date          8360 non-null   object 
 4   home_team_id       8360 non-null   object 
 5   away_team_id       8360 non-null   object 
 6   home_goals_scored  7600 non-null   float64
 7   away_goals_scored  7600 non-null   float64
 8   home_formation     8360 non-null   object 
 9   away_formation     8360 non-null   object 
 10  home_num_injuries  8360 non-null   int64  
 11  away_num_injuries  8360 non-null   int64  
 12  referee_id         8360 non-null   object 
dtypes: float64(2), int64(4), object(7)
memory usage: 914.4+ KB


In [146]:
df['Home Team Won'] = df.apply(lambda x : 1 if (x['home_goals_scored']>x['away_goals_scored']) else 0, axis=1)

In [147]:
df.head()

,season,game_week,game_id,game_date,home_team_id,away_team_id,home_goals_scored,away_goals_scored,home_formation,away_formation,home_num_injuries,away_num_injuries,referee_id,Home Team Won
0,2012,1,efl_00001,2012/02/08,xmq321,1xyza3,1.0,0.0,four three three,four two four,0,2,brit12,1
1,2012,1,efl_00002,2012/02/08,ytug99,jzxz42,2.0,3.0,four five one,four three three,0,0,kope3,0
2,2012,1,efl_00003,2012/02/08,9hgy&,uythe,2.0,3.0,four five one,four five one,1,4,xyyu5,0
3,2012,1,efl_00004,2012/02/08,6re#$3,bl4a#,2.0,0.0,four five one,three four three,0,3,manc77,1
4,2012,1,efl_00005,2012/02/08,gyu43a,krkbr33,2.0,3.0,four four two,four five one,0,0,xhh46,0


In [148]:
df['home_stadium_capacity'] = df.apply(lambda x : teams[teams['team_id']==x['home_team_id']]['stadium_capacity'].iloc[0], axis=1)

df['game_date'] = pd.to_datetime(df['game_date'])

df["home_formation_front"] = df['home_formation'].apply(lambda x : x.split()[0])
df['home_formation_mid'] = df['home_formation'].apply(lambda x : x.split()[1])
df["home_formation_back"] = df['home_formation'].apply(lambda x : x.split()[2])

df["away_formation_front"] = df['away_formation'].apply(lambda x : x.split()[0])
df['away_formation_mid'] = df['away_formation'].apply(lambda x : x.split()[1])
df["away_formation_back"] = df['away_formation'].apply(lambda x : x.split()[2])

In [149]:
cols_to_drop = ["home_formation","away_formation","game_id"]
new_df = df.drop(cols_to_drop, axis=1)

In [150]:
def make_num(x):
    if (x=="one"):
        return 1
    elif (x=="two"):
        return 2
    elif (x=="three"):
        return 3
    elif (x=="four"):
        return 4
    elif (x=="five"):
        return 5

In [151]:
new_df['home_formation_front'] = new_df['home_formation_front'].apply(lambda x : make_num(x))
new_df['home_formation_mid'] = new_df['home_formation_mid'].apply(lambda x :make_num(x))
new_df["home_formation_back"] = new_df['home_formation_back'].apply(lambda x : make_num(x))

new_df["away_formation_front"] = new_df['away_formation_front'].apply(lambda x : make_num(x))
new_df['away_formation_mid'] = new_df['away_formation_mid'].apply(lambda x : make_num(x))
new_df["away_formation_back"] = new_df['away_formation_back'].apply(lambda x : make_num(x))

new_df['referee_experience'] = new_df.apply(lambda x : referees[referees['referee_id']==x['referee_id']]['referee_experience'].iloc[0], axis=1)
new_df['referee_from_home'] = new_df.apply(lambda x : 1 if (teams[teams['team_id']==x['home_team_id']]['team_country'].iloc[0]==referees[referees['referee_id']==x['referee_id']]['referee_country'].iloc[0]) else 0, axis=1)

new_df['goal_difference'] = new_df['home_goals_scored'] - new_df['away_goals_scored']

In [152]:
new_df['goal_difference'].unique()

array([ 1., -1.,  2., -2., nan])

In [153]:
emay = pd.read_excel("./Data/emay.xlsx")
emay2 = pd.read_excel("./Data/Events_Data.xlsx",sheet_name='updated')
# emay.head()

In [154]:
emay2.head()

,season,home_team_id,Unnamed: 2,Unnamed: 3,away_team_id,Unnamed: 5,Away_manager_win_percent,home_manager_age,away_manager_age,Home_Mgr_home_win_percent,...,home_forward,home_mid,home_back,away_forward,away_mid,away_back,home_num_injuries,away_num_injuries,home_win,manager_win_percent
0,2012,xmq321,2012xmq321,doal30,1xyza3,20121xyza3,koas35,59,45,0.357616,...,4,3,3,4,2,4,0,2,1,NaN
1,2012,ytug99,2012ytug99,scon40,jzxz42,2012jzxz42,huud97,48,47,0.534392,...,4,5,1,4,3,3,0,0,0,NaN
2,2012,9hgy&,20129hgy&,alof44,uythe,2012uythe,mart16,64,55,0.826316,...,4,5,1,4,5,1,1,4,0,NaN
3,2012,6re#$3,20126re#$3,rani87,bl4a#,2012bl4a#,anry76,60,48,0.750831,...,4,5,1,3,4,3,0,3,1,NaN
4,2012,gyu43a,2012gyu43a,juer88,krkbr33,2012krkbr33,geri24,61,46,0.611111,...,4,4,2,4,5,1,0,0,0,NaN


In [155]:
new_df['home_manager_wins'] = emay["home team manager's win percentage"]
new_df['away_manager_wins'] = emay["Away team manager's win percentage"]

new_df['home_manager_age'] = emay2["home_manager_age"]
new_df['away_manager_age'] = emay2["away_manager_age"]


In [157]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8360 entries, 0 to 759
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   season                 8360 non-null   int64         
 1   game_week              8360 non-null   int64         
 2   game_date              8360 non-null   datetime64[ns]
 3   home_team_id           8360 non-null   object        
 4   away_team_id           8360 non-null   object        
 5   home_goals_scored      7600 non-null   float64       
 6   away_goals_scored      7600 non-null   float64       
 7   home_num_injuries      8360 non-null   int64         
 8   away_num_injuries      8360 non-null   int64         
 9   referee_id             8360 non-null   object        
 10  Home Team Won          8360 non-null   int64         
 11  home_stadium_capacity  8360 non-null   int64         
 12  home_formation_front   8360 non-null   int64         
 13  home

In [161]:
unusable_cols = ["game_date","home_team_id","away_team_id","referee_id","goal_difference","home_goals_scored","away_goals_scored"]
train = new_df.drop(unusable_cols,axis=1)

In [162]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8360 entries, 0 to 759
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   season                 8360 non-null   int64  
 1   game_week              8360 non-null   int64  
 2   home_num_injuries      8360 non-null   int64  
 3   away_num_injuries      8360 non-null   int64  
 4   Home Team Won          8360 non-null   int64  
 5   home_stadium_capacity  8360 non-null   int64  
 6   home_formation_front   8360 non-null   int64  
 7   home_formation_mid     8360 non-null   int64  
 8   home_formation_back    8360 non-null   int64  
 9   away_formation_front   8360 non-null   int64  
 10  away_formation_mid     8360 non-null   int64  
 11  away_formation_back    8360 non-null   int64  
 12  referee_experience     8360 non-null   int64  
 13  referee_from_home      8360 non-null   int64  
 14  home_manager_wins      8360 non-null   float64
 15  away_

In [163]:
train['Home Team Won'].value_counts()

1    4368
0    3992
Name: Home Team Won, dtype: int64

### Normalising the data

In [166]:
std = StandardScaler()
transformed = pd.DataFrame(std.fit_transform(train),columns=train.columns)

In [167]:
transformed.head()

,season,game_week,home_num_injuries,away_num_injuries,Home Team Won,home_stadium_capacity,home_formation_front,home_formation_mid,home_formation_back,away_formation_front,away_formation_mid,away_formation_back,referee_experience,referee_from_home,home_manager_wins,away_manager_wins,home_manager_age,away_manager_age
0,-1.581139,-1.709409,-0.885548,1.008643,0.955991,0.480388,-0.234447,-0.630158,0.695940,-0.235723,-1.725853,1.665765,-0.559770,3.230540,-2.105456,0.190937,0.987382,-1.061901
1,-1.581139,-1.709409,-0.885548,-0.874608,-1.046035,-0.271443,-0.234447,1.605886,-1.255446,-0.235723,-0.624805,0.695935,-0.559770,-0.309546,-0.341811,-1.481057,-0.622769,-0.769146
2,-1.581139,-1.709409,0.058735,2.891895,-1.046035,-0.878104,-0.234447,1.605886,-1.255446,-0.235723,1.577290,-1.243726,-1.323986,-0.309546,1.120343,0.587510,1.719268,0.401872
3,-1.581139,-1.709409,-0.885548,1.950269,0.955991,-1.587990,-0.234447,1.605886,-1.255446,-1.805959,0.476243,0.695935,0.586554,-0.309546,0.444342,1.052904,1.133759,-0.622769
4,-1.581139,-1.709409,-0.885548,-0.874608,-1.046035,-0.259268,-0.234447,0.487864,-0.279753,-0.235723,1.577290,-1.243726,0.968662,-0.309546,0.015946,1.219223,1.280136,-0.915524


In [169]:
final_train = transformed[:7600]
final_test = transformed[7600:]

In [241]:
X = final_train.drop("Home Team Won",axis=1)
# y = new_df['goal_difference']+2
y = train['Home Team Won'][:7600]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("training data -->",X_train.shape)
print("testing data -->",X_test.shape)

training data --> (6080, 17)
testing data --> (1520, 17)


In [171]:
y.value_counts()

1    4368
0    3232
Name: Home Team Won, dtype: int64

### Modelling

#### XGBoost

In [238]:
model = XGBClassifier()

model.fit(X_train, y_train)
y_preds = model.predict(X_test)

print(accuracy_score(y_test,y_preds))
print(confusion_matrix(y_test,y_preds))
print(classification_report(y_test,y_preds))

/Users/raahul/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/raahul/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[10:12:23] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_eek2t0c4ro/croots/recipe/xgboost-split_1659548960591/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.7447368421052631
[[554 254]
 [231 861]]
              precision    recall  f1-score   support

           0       0.71      0.69      0.70       808
           1       0.77      0.79      0.78      1092

    accuracy                           0.74      1900
   macro avg       0.74      0.74      0.74      1900
weighted avg       0.74      0.74      0.74      1900



In [133]:
# grid = {'iterations': [1000,6000],
#         'learning_rate': [0.05, 0.005, 0.0005],
#         'depth': [4, 6, 10],
#         'l2_leaf_reg': [1, 3, 5, 9]
#         }

# model = CatBoostClassifier()
# randomized_search_result = model.randomized_search(grid,
#                                                    X = X_train,
#                                                    y= y_train,
#                                                    verbose = False,
#                                                    plot=True)

In [83]:
# randomized_search_result['params']

{'depth': 4, 'l2_leaf_reg': 1, 'iterations': 6000, 'learning_rate': 0.005}

#### CatBoost

In [242]:
params = {'iterations': 6000,
          'learning_rate': 0.005,
          'depth': 4,
          'l2_leaf_reg': 1,
          'early_stopping_rounds': 200,
          'verbose':0,
          'random_seed': 42}

cat_model = CatBoostClassifier(**params)

cat_model.fit(X_train, y_train)
y_preds = cat_model.predict(X_test)

In [115]:
# y_test = y_test.apply(lambda x : 1 if(x>2) else 0)
# y_preds = pd.Series(y_preds[:,0]).apply(lambda x : 1 if(x>2) else 0)

In [243]:
print(accuracy_score(y_test,y_preds))
print(confusion_matrix(y_test,y_preds))
print(classification_report(y_test,y_preds))

0.7651315789473684
[[446 200]
 [157 717]]
              precision    recall  f1-score   support

           0       0.74      0.69      0.71       646
           1       0.78      0.82      0.80       874

    accuracy                           0.77      1520
   macro avg       0.76      0.76      0.76      1520
weighted avg       0.76      0.77      0.76      1520



#### Support Vector Classifier

In [215]:
model = SVC()

model.fit(X_train, y_train)
y_preds = model.predict(X_test)

print(accuracy_score(y_test,y_preds))
print(confusion_matrix(y_test,y_preds))
print(classification_report(y_test,y_preds))

0.6618421052631579
[[272 374]
 [140 734]]
              precision    recall  f1-score   support

           0       0.66      0.42      0.51       646
           1       0.66      0.84      0.74       874

    accuracy                           0.66      1520
   macro avg       0.66      0.63      0.63      1520
weighted avg       0.66      0.66      0.64      1520



## Prediction

In [196]:
karth = pd.read_excel("./Data/Karthik.xlsx",sheet_name=1)
karth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 15 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   season                              760 non-null    int64  
 1   game_week                           760 non-null    int64  
 2   game_id                             760 non-null    object 
 3   game_date                           760 non-null    object 
 4   home_team_id                        760 non-null    object 
 5   away_team_id                        760 non-null    object 
 6   home_formation                      760 non-null    object 
 7   away_formation                      760 non-null    object 
 8   home_num_injuries                   760 non-null    int64  
 9   away_num_injuries                   760 non-null    int64  
 10  referee_id                          760 non-null    object 
 11  home team manager's win percentage  760 non-n

In [201]:
final_test['home_manager_wins'].iloc[:] = karth['home team manager\'s win percentage']
final_test['away_manager_wins'].iloc[:] = karth['Away team manager\'s win percentage']

final_test['home_manager_age'].iloc[:] = karth['home_manager_age']
final_test['away_manager_age'].iloc[:] = karth['away_manager_age']

/var/folders/s4/rbw51hr572v983ntn5czj87h0000gn/T/ipykernel_2996/754731901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_test['home_manager_wins'].iloc[:] = karth['home team manager\'s win percentage']
/var/folders/s4/rbw51hr572v983ntn5czj87h0000gn/T/ipykernel_2996/754731901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_test['away_manager_wins'].iloc[:] = karth['Away team manager\'s win percentage']
/var/folders/s4/rbw51hr572v983ntn5czj87h0000gn/T/ipykernel_2996/754731901.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docum

In [204]:
final_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 7600 to 8359
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   season                 760 non-null    float64
 1   game_week              760 non-null    float64
 2   home_num_injuries      760 non-null    float64
 3   away_num_injuries      760 non-null    float64
 4   Home Team Won          760 non-null    float64
 5   home_stadium_capacity  760 non-null    float64
 6   home_formation_front   760 non-null    float64
 7   home_formation_mid     760 non-null    float64
 8   home_formation_back    760 non-null    float64
 9   away_formation_front   760 non-null    float64
 10  away_formation_mid     760 non-null    float64
 11  away_formation_back    760 non-null    float64
 12  referee_experience     760 non-null    float64
 13  referee_from_home      760 non-null    float64
 14  home_manager_wins      760 non-null    float64
 15  aw

In [244]:
final_prediction = cat_model.predict(final_test)
final_prediction_prob = cat_model.predict_proba(final_test)

#### Submission File

In [247]:
submission = pd.read_excel("./Submission_Template.xlsx")
submission.head()

,game_id,home_team_win_probability
0,efl_07601,NaN
1,efl_07602,NaN
2,efl_07603,NaN
3,efl_07604,NaN
4,efl_07605,NaN


In [250]:
submission['home_team_win_probability'] = np.round(final_prediction_prob[:,1],4)

In [251]:
submission

,game_id,home_team_win_probability
0,efl_07601,0.2968
1,efl_07602,0.2923
2,efl_07603,0.4586
3,efl_07604,0.4514
4,efl_07605,0.6514
...,...,...
755,efl_08356,0.7753
756,efl_08357,0.4342
757,efl_08358,0.8087
758,efl_08359,0.6540


In [253]:
submission.to_excel("final_submission.xlsx",index=False)